# Phase 3.5: HumanEval Temperature Robustness Results Inspection

This notebook inspects Phase 3.5 HumanEval results to verify:
- Dataset files created for each temperature
- metadata.json contains correct information
- Prompts formatted correctly
- Code generation working
- Evaluation results accurate
- Activations captured

In [1]:
import pandas as pd
import os
from pathlib import Path
import glob
import json

# Set pandas display options to show FULL content
pd.set_option('display.max_colwidth', None)  # Show full column content (NO LIMITS)
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', None)         # Don't wrap to multiple lines

print("✅ Pandas display options set to show FULL content")

✅ Pandas display options set to show FULL content


In [2]:
# Auto-discovery of Phase 3.5 HumanEval data
datasets_dir = "../data/phase3_5_humaneval/"
pattern = os.path.join(datasets_dir, "dataset_temp_*.parquet")
matching_files = glob.glob(pattern)

if matching_files:
    # Sort by temperature (filename)
    matching_files.sort()
    print("=" * 80)
    print("PHASE 3.5: HUMANEVAL TEMPERATURE ROBUSTNESS")
    print("=" * 80)
    print(f"\n🔍 Found {len(matching_files)} temperature file(s)")
    for file in matching_files:
        file_size = os.path.getsize(file) / 1024  # KB
        print(f"  📁 {Path(file).name} ({file_size:.2f} KB)")
else:
    raise FileNotFoundError(f"❌ No temperature dataset files found in {datasets_dir}")

PHASE 3.5: HUMANEVAL TEMPERATURE ROBUSTNESS

🔍 Found 1 temperature file(s)
  📁 dataset_temp_0_0.parquet (17.69 KB)


In [3]:
# Load and display metadata.json
metadata_path = Path(datasets_dir) / "metadata.json"

if metadata_path.exists():
    print("\n" + "=" * 80)
    print("METADATA.JSON")
    print("=" * 80)
    
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    print("\n📄 Full metadata content:")
    print(json.dumps(metadata, indent=2))
    
    print("\n📊 Key information:")
    print(f"  - Creation timestamp: {metadata.get('creation_timestamp', 'N/A')}")
    print(f"  - Best layers: Correct={metadata.get('best_layers', {}).get('correct', 'N/A')}, Incorrect={metadata.get('best_layers', {}).get('incorrect', 'N/A')}")
    print(f"  - Features: Correct={metadata.get('best_layers', {}).get('correct_feature_idx', 'N/A')}, Incorrect={metadata.get('best_layers', {}).get('incorrect_feature_idx', 'N/A')}")
    print(f"  - Temperatures tested: {metadata.get('temperatures', [])}")
    print(f"  - Tasks attempted: {metadata.get('n_tasks_attempted', 'N/A')}")
    print(f"  - Tasks included: {metadata.get('n_tasks_included', 'N/A')}")
else:
    print(f"\n⚠️  metadata.json not found at {metadata_path}")


METADATA.JSON

📄 Full metadata content:
{
  "creation_timestamp": "2025-11-21T06:16:33.238829",
  "best_layers": {
    "correct": 16,
    "incorrect": 19,
    "correct_feature_idx": 14439,
    "incorrect_feature_idx": 5441
  },
  "extraction_layers": [
    16,
    19
  ],
  "temperatures": [
    0.0
  ],
  "samples_per_temperature": 3,
  "validation_task_ids": [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10
  ],
  "n_tasks_attempted": 11,
  "n_tasks_included": 11,
  "n_tasks_excluded": 0,
  "exclusion_rate_percent": 0.0,
  "excluded_task_ids": [],
  "n_total_samples": 11,
  "temperature_stats": {
    "0.0": {
      "n_correct": 1,
      "n_incorrect": 10,
      "pass_rate": 0.09090909090909091,
      "avg_generation_time": 13.048784602772106
    }
  }
}

📊 Key information:
  - Creation timestamp: 2025-11-21T06:16:33.238829
  - Best layers: Correct=16, Incorrect=19
  - Features: Correct=14439, Incorrect=5441
  - Temperatures tested: [0.0]
  - Tasks attemp

In [4]:
# Check activations directory
activations_dir = Path(datasets_dir) / "activations"

print("\n" + "=" * 80)
print("ACTIVATIONS DIRECTORY")
print("=" * 80)

if activations_dir.exists():
    print(f"\n✅ Activations directory exists: {activations_dir}")
    
    # Check subdirectories
    subdirs = [d for d in activations_dir.iterdir() if d.is_dir()]
    print(f"\n📁 Subdirectories ({len(subdirs)}):")
    for subdir in subdirs:
        files = list(subdir.glob("*.npz"))
        print(f"  - {subdir.name}/: {len(files)} activation files")
        if files:
            print(f"    Sample: {files[0].name}")
else:
    print(f"\n⚠️  Activations directory not found: {activations_dir}")


ACTIVATIONS DIRECTORY

✅ Activations directory exists: ../data/phase3_5_humaneval/activations

📁 Subdirectories (1):
  - task_activations/: 22 activation files
    Sample: 3_layer_16.npz


In [5]:
# Load and display each temperature dataset
for file_path in matching_files:
    file_name = Path(file_path).name
    temp = file_name.replace('dataset_temp_', '').replace('.parquet', '').replace('_', '.')
    
    print(f"\n{'=' * 80}")
    print(f"TEMPERATURE: {temp}")
    print(f"Dataset: {file_name}")
    print("=" * 80)
    
    df = pd.read_parquet(file_path)
    
    # Basic stats
    print(f"\n📊 Statistics:")
    print(f"  - Total records: {len(df)}")
    print(f"  - Columns: {list(df.columns)}")
    
    # Pass rate
    if 'test_passed' in df.columns:
        n_passed = df['test_passed'].sum()
        pass_rate = n_passed / len(df) * 100
        print(f"\n✅ Pass Rate:")
        print(f"  - Passed: {n_passed}/{len(df)} ({pass_rate:.2f}%)")
        print(f"  - Failed: {len(df) - n_passed}/{len(df)} ({100 - pass_rate:.2f}%)")
    
    # Generation time stats
    if 'generation_time' in df.columns:
        print(f"\n⏱️  Generation Time:")
        print(f"  - Mean: {df['generation_time'].mean():.2f}s")
        print(f"  - Median: {df['generation_time'].median():.2f}s")
        print(f"  - Min: {df['generation_time'].min():.2f}s")
        print(f"  - Max: {df['generation_time'].max():.2f}s")
    
    print(f"\n{'=' * 80}")
    print(f"First 10 records:")
    print("=" * 80)
    display(df.head(10))


TEMPERATURE: 0.0
Dataset: dataset_temp_0_0.parquet

📊 Statistics:
  - Total records: 11
  - Columns: ['task_id', 'temperature', 'prompt', 'generated_code', 'test_passed', 'error_message', 'generation_time', 'cyclomatic_complexity', 'generation_idx', 'test_list']

✅ Pass Rate:
  - Passed: 1/11 (9.09%)
  - Failed: 10/11 (90.91%)

⏱️  Generation Time:
  - Mean: 13.05s
  - Median: 8.61s
  - Min: 2.25s
  - Max: 38.03s

First 10 records:


,task_id,temperature,prompt,generated_code,test_passed,error_message,generation_time,cyclomatic_complexity,generation_idx,test_list
0,0,0.0,"from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n """""" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n """"""\n\n\nassert assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\nassert assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\nassert assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\nassert assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\nassert assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\nassert assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\nassert assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n# Solution:","def has_close_elements(numbers: List[float], threshold: float) -> bool:\n """""" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n """"""\n for i in range(len(numbers)):\n for j in range(i+1, len(numbers)):\n if abs(numbers[i] - numbers[j]) < threshold:\n return True\n return False",False,None,8.913819,0,0,"[""assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True"", ""assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False"", ""assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True"", ""assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False"", ""assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True"", ""assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True"", ""assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False""]"
1,1,0.0,"from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n """""" Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n separate those group into separate strings and return the list of those.\n Separate groups are balanced (each open brace is properly closed) and not nested within each other\n Ignore any spaces in the input string.\n >>> separate_paren_groups('( ) (( )) (( )( ))')\n ['()', '(())', '(()())']\n """"""\n\n\nassert assert separate_paren_groups('(()()) ((())) () ((())()())') == [\nassert assert separate_paren_groups('() (()) ((())) (((())))') == [\nassert assert separate_paren_groups('(()(())((())))') == [\nassert assert separate_paren_groups('( ) (( )) (( )( ))') == ['()', '(())', '(()())']\n\n# Solution:","def separate_paren_groups(paren_string: str) -> List[str]:\n """""" Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n separate those group into separate strings and return the list of those.\n Separate groups are balanced (each open brace is properly closed) and not nested within each other\n Ignore any spaces in the input string.\n >>> separate_paren_groups('( ) (( )) (( )( ))')\n ['()', '(())', '(()())']\n """"""\n paren_string = paren_string.replace(' ', '')\n paren_string = paren_string.replace('(', '(')\n paren_string = paren_string.replace(')', ')')\n paren_string = paren_string.replace('()', '')\n paren_string = paren_string.replace('(())', '')\n paren_string = paren_string.replace('(()())', '')\n paren_string = paren_string.replace('(()()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_string.replace('(()', '')\n paren_string = paren_

In [ ]:
# Summary across all temperatures
print("\n" + "=" * 80)
print("OVERALL SUMMARY")
print("=" * 80)

print(f"\n📊 Files processed: {len(matching_files)}")
print(f"📁 Metadata exists: {'✅' if metadata_path.exists() else '❌'}")
print(f"📁 Activations exist: {'✅' if activations_dir.exists() else '❌'}")

print("\n🎉 Phase 3.5 HumanEval results inspection complete!")